In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

6604534


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

13209068
13203430


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
11303866,1112513555,GNZ6rRw,0,MASC,24.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,NaN
222903,1112281548,pz9RqRN,0,MASC,27.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
2557427,1112402611,8R3YQx,1,MASC,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,NaN
4113599,1112335211,NwMvRJ,1,FEM,28.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
11615078,1112468750,owEVdle,0,FEM,21.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
8126609,1112463373,wVjVLRV,0,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
7153847,1112365693,1469wQ,0,FEM,44.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,NaN
10005912,1112326525,dY5zDZP,0,MASC,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
8302938,1112235241,lD52wXj,0,FEM,23.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN
1684729,1112196813,8oV3bx,1,MASC,34.0,-,En Curso,-,Graduado,Graduado,...,False,False,False,False,True,True,False,False,False,0.009615


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
1413484,1112033906,MVPAlLo,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.003044
8990742,1112444023,ZDPpPBV,1,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,0.051282
8494422,1112254343,5mrj99z,0,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,True,False,0.000000
8020644,1112273003,VNrYzrd,1,MASC,20.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
11232677,1112431057,ak5w4OZ,0,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
6971693,1112296029,5JBQrq,0,MASC,41.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.000000
9531775,1111284383,BmZk6R2,0,FEM,46.0,-,Graduado,Graduado,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.000000
8355729,1112260331,8Poaw3,1,FEM,33.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
8367465,1112284454,KBa1Qxj,1,FEM,21.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
888519,1112299531,5mP9GBv,1,FEM,33.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000


In [15]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Administración,1
1,0z5Dmrd,Almacén / Depósito / Expedición,1
2,0z5Dmrd,Atención al Cliente,1
3,0z5Dmrd,Auditoría,1
4,0z5Dmrd,Contabilidad,1


In [16]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
1,0z5Dmrd,Junior,2
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,13
4,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2


In [17]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,16
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,21
3,0z5VvGv,Full-time,50
4,0z5VvGv,Part-time,10


In [18]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
11888777,1112254717,W9Wja8j,0,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,0.000000,2,57,48
12066142,1112117980,59xb2v,0,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,0,15,9
11742144,1111685518,X9k8oav,0,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,0.000000,0,4,6
9347439,1112377301,X9k4YPx,1,FEM,30.0,-,-,-,-,-,...,False,False,False,False,False,False,0.000000,0,14,9
3820585,1112285761,8APRox,1,MASC,26.0,-,-,-,-,-,...,False,True,True,False,False,False,0.000000,98,702,515
12281976,1111013186,A3XX1lL,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,0.000000,0,12,7
10949639,1112404221,ak5PWY5,1,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,29,141,119
948220,1112109888,mzjpQad,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.005038,10,0,13
1629652,1112460021,EZAZKz,1,MASC,58.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.007937,32,54,4
11644841,1112408780,EzEM5E4,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,4,49,41


In [19]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  12148226 Test:  639381


In [20]:
gc.collect()

92

In [21]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.687243
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.680494
[3]	valid_0's binary_logloss: 0.673562
[4]	valid_0's binary_logloss: 0.666785
[5]	valid_0's binary_logloss: 0.661226
[6]	valid_0's binary_logloss: 0.654655
[7]	valid_0's binary_logloss: 0.648517
[8]	valid_0's binary_logloss: 0.642187
[9]	valid_0's binary_logloss: 0.636006
[10]	valid_0's binary_logloss: 0.630024
[11]	valid_0's binary_logloss: 0.624454
[12]	valid_0's binary_logloss: 0.618595
[13]	valid_0's binary_logloss: 0.613366
[14]	valid_0's binary_logloss: 0.607878
[15]	valid_0's binary_logloss: 0.60263
[16]	valid_0's binary_logloss: 0.597552
[17]	valid_0's binary_logloss: 0.592141
[18]	valid_0's binary_logloss: 0.587337
[19]	valid_0's binary_logloss: 0.582624
[20]	valid_0's binary_logloss: 0.577878
[21]	valid_0's binary_logloss: 0.572807
[22]	valid_0's binary_logloss: 0.56785
[23]	valid_0's binary_logloss: 0.563271
[24]	valid_0's binary_logloss:

In [39]:
score=c.roc_auc_score(x_test,y_test)*100
print(score)

AttributeError: 'LGBMClassifier' object has no attribute 'roc_auc_score'

In [23]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV18.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV18.pkl']

In [25]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
57149,62362,1112456072,N01KLJ,MASC,33.0,-,-,-,-,-,...,False,False,False,False,False,False,0.000000,0.0,13.0,9.0
41016,29541,1112372247,lDLGxo6,MASC,19.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,0.000000,1.0,11.0,10.0
51666,20142,1112248723,8k0Kql,FEM,32.0,-,-,En Curso,-,-,...,False,True,False,False,False,False,0.000000,0.0,34.0,0.0
33704,45606,1112428517,wVkARkm,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.026316,16.0,49.0,39.0
47075,84361,1112466548,zv6BjOJ,MASC,33.0,-,-,Graduado,-,Graduado,...,False,False,False,True,False,True,0.000000,171.0,400.0,275.0
31622,34807,1112409290,xk94VO0,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,True,False,True,0.000000,18.0,29.0,21.0
39804,63162,1112457309,wVYN9RZ,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,1.0,8.0,7.0
67110,13280,1112034433,YWreBY,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,0.000000,0.0,1.0,2.0
68715,52102,1112442467,6krvp4,FEM,42.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,0.000000,0.0,1.0,0.0
36661,85642,1112466646,kP6VEaQ,MASC,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,0.031250,0.0,13.0,3.0


In [26]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.98468452, 0.01531548],
       [0.91707531, 0.08292469],
       [0.99507161, 0.00492839],
       ...,
       [0.98484307, 0.01515693],
       [0.98797481, 0.01202519],
       [0.99410825, 0.00589175]])

In [27]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [28]:
prediccion['sepostulo']=y_final2

In [29]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,0.0,0.0,13.0,0.0,0.015315
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,0.0,5.0,55.0,0.0,0.082925
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,0.0,2.0,10.0,0.0,0.004928
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,0.0,0.0,12.0,0.0,0.002338
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,0.0,0.0,10.0,0.0,0.000976


In [30]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.015315
1,1,0.082925
2,2,0.004928
3,3,0.002338
4,4,0.000976


In [31]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion36.csv",index=False)

In [32]:
prediccion.sepostulo.mean()

0.4772061416245788